In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("marcelloVendruscolo_A1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

In [22]:
#A.1.1 - Read the English transcripts with Spark, and count the number of lines.
print("File: europarl-v7.sv-en.en")
file_eng = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en")
line_count_eng = file_eng.count()
print("Line counting: " + str(line_count_eng))

File: europarl-v7.sv-en.en
Line counting: 1862234


In [23]:
#A.1.2 - Read the Swedish transcripts with Spark, and count the number of lines.
print("File: europarl-v7.sv-en.sv")
file_sv = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv")
line_count_sv = file_sv.count()
print("Line counting: " + str(line_count_sv))


File: europarl-v7.sv-en.sv
Line counting: 1862234


In [24]:
#A.1.3 - Verify that the line counts are the same for the two languages.
if (line_count_eng == line_count_sv):
    print("The line counts are EQUAL for the two languages.")
else:
    print("The line counts are DIFFERENT for the two languages.")

The line counts are EQUAL for the two languages.


In [25]:
#A.1.4 Count the number of partitions.
print("Partition counting: " + str(file_eng.getNumPartitions()))
print("Partition counting: " + str(file_sv.getNumPartitions()))

Partition counting: 2
Partition counting: 3


In [26]:
# release the cores for another application!
#spark_context.stop()